In [1]:
import keras as k
import os
from keras.utils import *


Using TensorFlow backend.


In [2]:
from keras.layers import Input,Dense,Conv2D,MaxPooling2D,Flatten,UpSampling2D,Dropout,Conv2DTranspose,Cropping2D
from keras.models import Model

df='channels_first'

In [3]:
def FCN16(n_classes,h=512,w=512,vgg_level=3):
    # Defining Input Layer
    inputs=Input(shape=(3,h,w))
    # Adding layers in Block 1
    x=Conv2D(64,(3,3),activation='relu',padding='same',name='block1_conv1',data_format=df)(inputs)
    x=Conv2D(64,(3,3),activation='relu',padding='same',name='block1_conv2',data_format=df)(x)
    x=MaxPooling2D((2,2),strides=(2,2),name='block1_pool')(x)
    f1=x
    # Adding layers in Block 2
    x=Conv2D(128,(3,3),activation='relu',padding='same',name='block2_conv1',data_format=df)(x)
    x=Conv2D(128,(3,3),activation='relu',padding='same',name='block2_conv2',data_format=df)(x)
    x=MaxPooling2D((2,2),strides=(2,2),name='block2_pool')(x)
    f2=x
    # Adding layers in Block 3
    x=Conv2D(256,(3,3),activation='relu',padding='same',name='block3_conv1',data_format=df)(x)
    x=Conv2D(256,(3,3),activation='relu',padding='same',name='block3_conv2',data_format=df)(x)
    x=Conv2D(256,(3,3),activation='relu',padding='same',name='block3_conv3',data_format=df)(x)
    x=MaxPooling2D((2,2),strides=(2,2),name='block3_pool',data_format=df)(x)
    f3=x
    # Adding layers in Block 4
    x=Conv2D(512,(3,3),activation='relu',padding='same',name='block4_conv1',data_format=df)(x)
    x=Conv2D(512,(3,3),activation='relu',padding='same',name='block4_conv2',data_format=df)(x)
    x=Conv2D(512,(3,3),activation='relu',padding='same',name='block4_conv3',data_format=df)(x)
    x=MaxPooling2D((2,2),strides=(2,2),name='block4_pool',data_format=df)(x)
    f4=x
    # Adding Layers in Block 5
    x=Conv2D(512,(3,3),activation='relu',padding='same',name='block5_conv1',data_format=df)(x)
    x=Conv2D(512,(3,3),activation='relu',padding='same',name='block5_conv2',data_format=df)(x)
    x=Conv2D(512,(3,3),activation='relu',padding='same',name='block5_conv3',data_format=df)(x)
    x=MaxPooling2D((2,2),strides=(2,2),name='block5_pool',data_format=df)(x)
    f5=x
    
    
    
    
    out=f5
    
    out=(Conv2D(4096,(7,7),activation='relu',padding='same',data_format=df))(out)
    out=Dropout(0.5)(out)
    out=(Conv2D(4096,(1,1),activation='relu',padding='same',data_format=df))(out)
    out=Dropout(0.5)(out)
    out=(Conv2D(n_classes,(1,1),kernel_initializer='he_normal',data_format=df))(out)
    out=(Conv2DTranspose(n_classes,kernel_size=(4,4),strides=(2,2),padding='valid',activation=None,name='score_2',data_format=df))(out)
    out=(Cropping2D(cropping=((0,2),(0,2))))(out)
   
    
    
    model=Model(inputs,out)
    Conv_size=model.layers[-2].output_shape[2]
    Deconv_size=model.layers[-1].output_shape[2]
    
    
    print(Conv_size,Deconv_size)
    
    Extra=Deconv_size-Conv_size
    
    return model


    
fcn_16=FCN16(21)
    

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'    
    
    
    
    
    
    

34 32


In [4]:
from keras.utils import plot_model

In [5]:
plot_model(fcn_16,to_file="FCN.png",show_shapes=True)

In [6]:
weights_path = os.path.expanduser(os.path.join('~', '.keras/models/vgg16_weights.h5'))


In [7]:
fcn_16.load_weights(weights_path, by_name=True)